In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def ECEF_to_WGS84(x, y, z):
    a = 6378137.0
    b = 6356752.31424518
    c = np.sqrt(((a * a) - (b * b)) / (a * a))
    d = np.sqrt(((a * a) - (b * b)) / (b * b))
    p = np.sqrt((x * x) + (y * y))
    q = np.arctan2((z * a), (p * b))
    Longitude = np.arctan2(y, x)
    Latitude = np.arctan2((z + (d * d) * b * np.power(np.sin(q), 3)), (p - (c * c) * a * np.power(np.cos(q), 3)))
    N = a / np.sqrt(1 - ((c * c) * np.power(np.sin(Latitude), 2)))
    Altitude = (p / np.cos(Latitude)) - N
    Longitude = Longitude * 180.0 / np.pi
    Latitude = Latitude * 180.0 / np.pi
    return Latitude, Longitude, Altitude

In [3]:
def dis(lon1, lat1, lon2, lat2):
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    ang = np.sin(np.deg2rad(dlat)/2)**2 + np.cos(np.deg2rad(lat1)) * np.cos(np.deg2rad(lat2)) * np.sin(np.deg2rad(dlon/2))**2
    c = 2. * np.arcsin(np.sqrt(ang))
    r = 6367000
    return c * r

In [4]:
def IsTooLarge(lat, log, SinLat, SinLog, A):
    index = []
    for j in range(lat.shape[0]):
        if lat[j] < SinLat - A:
            pass
        elif lat[j] < SinLat + A:
            index.append(j)
        else:
            break
    j = 0
    while j < len(index):
        if SinLog - A < log[index[j]] < SinLog + A:
            j += 1
        else:
            index.pop(j)

    if len(index) == 0:
        return True, index
    else:
        return False, index

In [5]:
def FindRoad(lat, log, SinLat, SinLog):
    for i in range(5, 2001, 5):
        flag, index = IsTooLarge(lat, log, SinLat, SinLog, i * 1e-6)
        if not flag:
            # print(i * 1e-2)
            return index, i
    return None, None

In [6]:
def RMM(RawLat: np.ndarray, RawLog: np.ndarray, A: float):
    map_data = 'road.csv'
    MD = pd.read_csv(map_data)
    lat = np.array(MD.iloc[:, 0])
    log = np.array(MD.iloc[:, 1])

    too_large_error = []
    for i in range(RawLat.shape[0]):
        SinLat = RawLat[i]
        SinLog = RawLog[i]

        if IsTooLarge(lat, log, SinLat, SinLog, A)[0]:
            print('选定范围内无道路数据点！')
            print('=' * 50)
            too_large_error.append(i)
        else:
            print('有可匹配数据点')
            # NewSinLat = 0
            # NewSinLog = 0
            # num = 0
            # for j in IsTooLarge(lat, log, SinLat, SinLog, A)[1]:
            #     NewSinLat += (1. / dis(SinLog, SinLat, lat[j], log[j])) * lat[j]
            #     NewSinLog += (1. / dis(SinLog, SinLat, lat[j], log[j])) * log[j]
            #     num += 1. / dis(SinLog, SinLat, lat[j], log[j])
            #
            # RawLat[i] = NewSinLat / num
            # RawLog[i] = NewSinLog / num
            print('=' * 50)

    # print(too_large_error)
    # print(len(too_large_error))

    isolated = []
    continuous = []
    for i in too_large_error:
        if (i - 1) in too_large_error or (i + 1) in too_large_error:
            continuous.append(i)
        else:
            isolated.append(i)
    # print(isolated)
    print('孤立误差点共', len(isolated))
    # print(continuous)
    print('连续误差点共', len(continuous))

    for i in isolated:
        RawLat[i] = (RawLat[i - 1] + RawLat[i + 1])/2.
        RawLog[i] = (RawLog[i - 1] + RawLog[i + 1])/2.
        # matched_error[i] = dis(RawLog[i], RawLat[i], Log[i], Lat[i])

    matched = []
    for i in continuous:
        # if (i + 1) in continuous:
            RoadData, a = FindRoad(lat, log, RawLat[i], RawLog[i])
            if a != None:
                # print(a)
                mid = int(len(RoadData) / 2)
                RawLat[i] = lat[RoadData[mid]]
                RawLog[i] = log[RoadData[mid]]
                # ang = np.arctan((RawLat[i] - lat[RoadData[mid]]) / (RawLog[i] - log[RoadData[mid]]))
                # ang = np.arctan(-1 / (lat[RoadData[-1]] - lat[RoadData[0]]) / (log[RoadData[-1]] - log[RoadData[0]]))
                # ang = np.arctan(-1 / (lat[i+1] - lat[i]) / (log[i+1] - log[i]))
                # for j in range(int(-a * 1.4), int(a * 1.4)):
                #     l = j * 1e-6
                #     x = l * np.cos(ang)
                #     y = l * np.sin(ang)
                #
                #     if not IsTooLarge(lat, log, RawLat[i]+y, RawLog[i]+x, A)[0]:
                #         RawLat[i] += y
                #         RawLog[i] += x
                #         # print(l)
                #         print('进度:', continuous.index(i)+1, '/', len(continuous), '匹配成功')
                #         break
                print('进度:', continuous.index(i)+1, '/', len(continuous), '匹配成功')
            else: print('进度:', continuous.index(i)+1, '/', len(continuous), '匹配失败')
        # else:
        #     RawLat[i] = (RawLat[i - 1] + RawLat[i + 1])/2.
        #     RawLog[i] = (RawLog[i - 1] + RawLog[i + 1])/2.
        #     print('进度:', continuous.index(i)+1, '/', len(continuous), '匹配成功')
    # print(len(matched))
    # for i in range(RawLat.shape[0]):
    #     print(RawLog)
    #     print(RawLat)

In [7]:
def M2D(array: np.ndarray):
    output = np.copy(array)
    for i in range(output.shape[0]):
        output[i] = int(array[i] / 100) + array[i] % 100 / 60.
    return output

In [8]:
DF = pd.read_csv('test.csv')

GPGGA = DF[DF['data form'].isin(['$GPGGA'])]
# GPGGA = GPGGA[GPGGA['GPS Status'].isin([4])]

RawLat = M2D(np.array(GPGGA['Latitude']))
RawLog = -M2D(np.array(GPGGA['Longitude']))

print(RawLog)

[-122.07916033 -122.0792315  -122.07921617 ... -122.0784315  -122.07843683
 -122.07843717]


In [9]:
import folium

# define the national map
city_map = folium.Map(location=[37.3502729, -121.993739], zoom_start=11)

for i in range(RawLat.shape[0]):
    # print(RawLat[i], ',', RawLog[i])

    folium.Circle(
        radius=5,
        location=(RawLat[i], RawLog[i]),
        # popup='Laurelhurst Park',
        color='#ff0000',
        fill=True,
        fill_color='#ff0000'
    ).add_to(city_map)

city_map.save('initial.html')

In [10]:
RMM(RawLat, RawLog, 5e-5)

for i in range(RawLat.shape[0]):
    # print(RawLat[i], ',', RawLog[i])

    folium.Circle(
        radius=5,
        location=(RawLat[i], RawLog[i]),
        # popup='Laurelhurst Park',
        color='#000000',
        fill=True,
        fill_color='#000000'
    ).add_to(city_map)

city_map.save('matched.html')

# print(RawLog)

选定范围内无道路数据点！
有可匹配数据点
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
选定范围内无道路数据点！
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有可匹配数据点
有